In [32]:
import sympy
import json
from pathlib import Path
rule_path = "lightning_logs/28/03/25 12:31 Simple_Saturation mlp/CV_1/explanations_cm.txt"
# parse json
NUM_STATES = 3
BOUNDING_PARAMETER = 10
explanations = json.loads(Path(rule_path).read_text())
explanations

{'input': {'y_0': [{'rule': 'c_0 & ~c_1 & ~c_2', 'count': 1980, 'index': 2}],
  'y_1': [{'rule': '~c_0 & ~c_1 & c_2', 'count': 10, 'index': 1}],
  'y_2': [{'rule': '~c_0 & c_1 & ~c_2', 'count': 10, 'index': 1}]},
 'StoneAgeLayer-0': {'y_0': [{'rule': 's_0 & ~s_1 & ~s_2 & s_0_count>0 & s_0_count>1 & s_0_count>2 & s_0_count>3 & s_0_count>4 & s_0_count>5 & s_0_count>6 & s_0_count>7 & s_0_count>8 & s_1_count>0 & s_2_count>0 & s_2_count>1 & s_2_count>2 & s_2_count>3 & s_2_count>4 & s_2_count>5 & s_2_count>6 & s_2_count>7 & ~s_1_count>s_0_count & ~s_2_count>s_0_count',
    'count': 594,
    'index': 2}],
  'y_1': [{'rule': 's_0 & ~s_1 & ~s_2 & s_0_count>0 & s_0_count>1 & s_0_count>2 & s_0_count>3 & s_0_count>4 & s_0_count>5 & s_0_count>6 & s_0_count>7 & s_0_count>8 & s_1_count>0 & s_1_count>1 & s_1_count>2 & s_1_count>3 & s_1_count>4 & s_1_count>5 & s_2_count>0 & s_2_count>1 & s_2_count>2 & s_2_count>3 & s_2_count>4 & s_2_count>5 & s_2_count>6 & ~s_2_count>7 & ~s_2_count>8 & ~s_2_count>9 & s

In [ ]:
import sympy
from itertools import product

from sympy import Implies
from sympy import simplify_logic
def simplify(expr):
    """
    Simplifies a sympy expression.
    """
    # Remove redundant parentheses
    # states are one_hot encoded
    assumptions = []
    for i,j in product(range(NUM_STATES), repeat=2):
        if i >= j:
            continue
        # remove redundant logical operators
        formula = f"~c_{i} |  ~c_{j}"
        formula = sympy.sympify(formula)
        assumptions.append(formula)
        formula = f"~(c_{i} & c_{j})"
        formula = sympy.sympify(formula)
        assumptions.append(formula)

    assumptions = sympy.And(*assumptions)
    #print("ASSUMPTIONS", assumptions)
    assumptions = sympy.simplify(assumptions)
    #print("ASSUMPTIONS", assumptions)
    expr = simplify_logic(Implies(assumptions, expr))
    #expr = sympy.refine(expr, assumptions)
    #expr = sympy.simplify(expr)
    # Remove redundant logical operators
    #expr = sympy.simplify_logic(expr)
    #print("S", expr)
    return expr
#rule = "c_0 & ~c_1 & ~c2"
#rule = "s_0 & ~s_1 & ~s_2 & s_0_count>0 & s_0_count>1".replace(">", "le")
per_y_rules = {}
for layer, rulebook in explanations.items():
    per_y_rules[layer] = {}

    for key, rules in rulebook.items():
        print(">", rules)
        agg_rules = [] 
        per_y_rules[layer][key] = {}
        for rule in rules:
            parsed = sympy.sympify(rule["rule"].replace(">", "le"))
            agg_rules.append(parsed)
        print(agg_rules)
        disjoint_rules = sympy.Or(*agg_rules)
        simplified_rule = simplify(disjoint_rules)
        per_y_rules[layer][key] = simplified_rule
    #raise Exception("stop")

# parse rule with sympy
#parsed = sympy.sympify(rule)
#parsed
per_y_rules

> [{'rule': 'c_0 & ~c_1 & ~c_2', 'count': 1980, 'index': 2}]
[c_0 & ~c_1 & ~c_2]
> [{'rule': '~c_0 & ~c_1 & c_2', 'count': 10, 'index': 1}]
[c_2 & ~c_0 & ~c_1]
> [{'rule': '~c_0 & c_1 & ~c_2', 'count': 10, 'index': 1}]
[c_1 & ~c_0 & ~c_2]


Exception: stop

In [46]:
from sympy import symbols, Not, simplify
from sympy.logic.boolalg import And, Or

# Define Boolean symbols
c0, c1, c2 = symbols('c0 c1 c2')

# Given Boolean expression
expr = And(c0, Not(c1), Not(c2))

# Additional known constraint
constraint = And(Or(Not(c0), Not(c1)), Or(Not(c0), Not(c2)), Or(Not(c1), Not(c2)))

# Simplify the expression under the given constraint
simplified_expr = simplify(And(expr, constraint))

print(simplified_expr)  # Expected output: c0


c0 & ~c1 & ~c2


In [47]:
from sympy import symbols, Not
from sympy.logic.boolalg import And, Or, Implies, simplify_logic

# Define Boolean symbols
c0, c1, c2 = symbols('c0 c1 c2')

# Given Boolean expression
expr = And(c0, Not(c1), Not(c2))

# Additional known constraint
constraint = And(Or(Not(c0), Not(c1)), Or(Not(c0), Not(c2)), Or(Not(c1), Not(c2)))

# Implication: If the constraint holds, then expr simplifies
simplified_expr = simplify_logic(Implies(constraint, expr))

print(simplified_expr)  # Expected output: c0


c0 | (c1 & c2)
